# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [59]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
from mysecrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [7]:
symbol = 'AMD'
api_url = f'https://api.iex.cloud/v1//data/core/QUOTE/{symbol}?token={IEX_CLOUD_API_TOKEN}'

data = requests.get(api_url).json()

data

[{'avgTotalVolume': 59840978,
  'calculationPrice': 'tops',
  'change': -3.64,
  'changePercent': -0.02451,
  'close': None,
  'closeSource': 'official',
  'closeTime': None,
  'companyName': 'Advanced Micro Devices Inc.',
  'currency': 'USD',
  'delayedPrice': None,
  'delayedPriceTime': None,
  'extendedChange': None,
  'extendedChangePercent': None,
  'extendedPrice': None,
  'extendedPriceTime': None,
  'high': None,
  'highSource': None,
  'highTime': None,
  'iexAskPrice': 148,
  'iexAskSize': 100,
  'iexBidPrice': 144.88,
  'iexBidSize': 100,
  'iexClose': 144.9,
  'iexCloseTime': 1704993402470,
  'iexLastUpdated': 1704993402470,
  'iexMarketPercent': 0.005924529307561135,
  'iexOpen': 148.64,
  'iexOpenTime': 1704983400492,
  'iexRealtimePrice': 144.9,
  'iexRealtimeSize': 92,
  'iexVolume': 206522,
  'lastTradeTime': 1704993402470,
  'latestPrice': 144.9,
  'latestSource': 'IEX real time price',
  'latestTime': '12:16:42 PM',
  'latestUpdate': 1704993402470,
  'latestVolume': 

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [10]:
peRatio = data[0]["peRatio"]
price = data[0]["latestPrice"]

peRatio

1129.38

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [11]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [18]:
final_dataframe = pd.DataFrame( columns= my_columns)

for symbol_string in symbol_strings:
    api_url = f'https://api.iex.cloud/v1//data/core/QUOTE/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    for i in range(0,len(data)):
        final_dataframe = final_dataframe._append(
            pd.Series(
                [
                    data[i]["symbol"],
                    data[i]["latestPrice"],
                    data[i]["peRatio"],
                    "N/A"
                ], index= my_columns
            ), ignore_index= True
        )


final_dataframe

/var/folders/tt/ptcp60n908q2mwfffgwm6jlm0000gn/T/ipykernel_1808/1330677335.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe = final_dataframe._append(
/var/folders/tt/ptcp60n908q2mwfffgwm6jlm0000gn/T/ipykernel_1808/1330677335.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe = final_dataframe._append(
/var/folders/tt/ptcp60n908q2mwfffgwm6jlm0000gn/T/ipykernel_1808/1330677335.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA ent

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,129.385,30.88,N/A
1,AAL,14.590,5.44,N/A
2,AAP,61.750,19.79,N/A
3,AAPL,184.725,30.13,N/A
4,ABBV,163.230,44.72,N/A
...,...,...,...,...
500,YUM,128.350,24.40,N/A
501,ZBH,122.610,54.25,N/A
502,ZBRA,251.490,28.04,N/A
503,ZION,42.755,4.56,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [19]:
final_dataframe.sort_values(my_columns[2], ascending=False, inplace=True)
final_dataframe = final_dataframe[final_dataframe[my_columns[2]] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace= True)
final_dataframe

,index,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,474,VTR,47.895,4024.79,N/A
1,31,AMD,147.100,1146.53,N/A
2,238,IFF,79.760,969.14,N/A
3,493,WYNN,93.535,317.50,N/A
4,273,KSU,293.590,279.61,N/A
5,97,CLX,141.810,205.08,N/A
6,481,WELL,90.075,179.47,N/A
7,208,GPS,20.900,175.48,N/A
8,77,CAH,105.275,172.92,N/A
9,150,DXCM,129.490,146.90,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [20]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [21]:
portfolio_input()

Enter the value of your portfolio: 1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [24]:
position_size = float(portfolio_size)/len(final_dataframe.index)

for row in final_dataframe.index:
    final_dataframe.loc[row,my_columns[3]] = math.floor(position_size/final_dataframe.loc[row, my_columns[1]])

final_dataframe

,index,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,474,VTR,47.895,4024.79,417
1,31,AMD,147.100,1146.53,135
2,238,IFF,79.760,969.14,250
3,493,WYNN,93.535,317.50,213
4,273,KSU,293.590,279.61,68
5,97,CLX,141.810,205.08,141
6,481,WELL,90.075,179.47,222
7,208,GPS,20.900,175.48,956
8,77,CAH,105.275,172.92,189
9,150,DXCM,129.490,146.90,154


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [32]:
symbol = 'AMD'

batch_api_call_url = f'https://api.iex.cloud/v1/data/CORE/ADVANCED_STATS/{symbol}?token={IEX_CLOUD_API_TOKEN}'

data = requests.get(batch_api_call_url).json()


# peRatio = data[0]["peRatio"]

# priceToBook = data[0]["priceToBook"]
# priceToSales = data[0]["priceToSales"]
# enterpriseValue = data[0]["enterpriseValue"]
# ebitda = data[0]["EBITDA"]

# ev_by_ebitda = enterpriseValue/ebitda

# grossProfit = data[0]["grossProfit"]

# ev_by_grossProfit = enterpriseValue/grossProfit



data



[{'beta': 2.000427091397583,
  'totalCash': 5785000000,
  'currentDebt': 7627000000,
  'revenue': 22111000000,
  'grossProfit': 11122000000,
  'totalRevenue': 22111000000,
  'EBITDA': 3110000000,
  'revenuePerShare': 13.82,
  'revenuePerEmployee': 1939561.4,
  'debtToEquity': 1.23023467345825,
  'profitMargin': 0.00940708244765049,
  'enterpriseValue': 233514000000,
  'enterpriseValueToRevenue': 10.56,
  'priceToSales': 10.71,
  'priceToBook': 4.31,
  'forwardPERatio': 34.09289802292882,
  'pegRatio': -12.418448111858632,
  'peHigh': 103.30587537091976,
  'peLow': 30.08878338278928,
  'week52highDate': '2023-12-29',
  'week52lowDate': '2023-01-19',
  'putCallRatio': 0.6939612501767783,
  'companyName': 'Advanced Micro Devices Inc.',
  'marketcap': 236832000000,
  'week52high': 151.05,
  'week52low': 67.27,
  'week52highSplitAdjustOnly': 151.05,
  'week52highDateSplitAdjustOnly': '2023-12-29',
  'week52lowSplitAdjustOnly': 67.27,
  'week52lowDateSplitAdjustOnly': '2023-01-19',
  'week52

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [35]:
column_ticker = 'Ticker'
column_price = 'Price'
column_shares_to_buy = 'Number of Shares to Buy'
column_peRatio = 'Price-to-Earnings Ratio'
column_pePercentile = 'PE Percentile'
column_priceToBook = 'Price-to-Book Ratio'
column_pbPercentile = 'PB Percentile'
column_priceToSales = 'Price-to-Sales Ratio'
column_psPercentile = 'PS Percentile'
column_ev_by_ebitda = 'EV/EBITDA'
column_ev_by_ebitda_percentile = 'EV/EBITDA Percentile'
column_ev_by_gp = 'EV/GP'
column_ev_by_gp_percentile = 'EV/GP Percentile'
column_rv_score = 'RV Score'

rv_columns = [
    column_ticker,
    column_price,
    column_shares_to_buy,
    column_peRatio,
    column_pePercentile,
    column_priceToBook,
    column_pbPercentile,
    column_priceToSales,
    column_psPercentile,
    column_ev_by_ebitda,
    column_ev_by_ebitda_percentile,
    column_ev_by_gp,
    column_ev_by_gp_percentile,
    column_rv_score
]


rv_dataframe = pd.DataFrame(columns= rv_columns)



for symbol_string in symbol_strings:
    stats_api_call_url = f'https://api.iex.cloud/v1/data/CORE/ADVANCED_STATS/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    basic_data_api_call_url = f'https://api.iex.cloud/v1//data/core/QUOTE/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'

    basic_data = requests.get(basic_data_api_call_url).json()
    stats_data = requests.get(stats_api_call_url).json()

    for row in range(0,len(basic_data)):

        enterpriseValue = stats_data[row]["enterpriseValue"]
        ebitda = stats_data[row]["EBITDA"]
        grossProfit = stats_data[row]["grossProfit"]

        try:
            ev_by_ebitda = enterpriseValue/ebitda

        except TypeError:
            ev_by_ebitda = np.NaN


        try:
            ev_by_grossProfit = enterpriseValue/grossProfit

        except TypeError:
            ev_by_grossProfit = np.NaN

        

        
        rv_dataframe = rv_dataframe._append(
            pd.Series(
                [
                    basic_data[row]["symbol"],
                    basic_data[row]["latestPrice"],
                    "N/A",
                    stats_data[row]["peRatio"],
                    "N/A",
                    stats_data[row]["priceToBook"],
                    "N/A",
                    stats_data[row]["priceToSales"],
                    "N/A",
                    ev_by_ebitda,
                    "N/A",
                    ev_by_grossProfit,
                    "N/A",
                    "N/A"
                    
                ], index= rv_columns
            ), ignore_index= True
        )


rv_dataframe
    

/var/folders/tt/ptcp60n908q2mwfffgwm6jlm0000gn/T/ipykernel_1808/2468288269.py:67: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  rv_dataframe = rv_dataframe._append(
/var/folders/tt/ptcp60n908q2mwfffgwm6jlm0000gn/T/ipykernel_1808/2468288269.py:67: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  rv_dataframe = rv_dataframe._append(
/var/folders/tt/ptcp60n908q2mwfffgwm6jlm0000gn/T/ipykernel_1808/2468288269.py:67: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is d

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,130.670,N/A,30.642558,N/A,6.50,N/A,5.5600,N/A,24.146682,N/A,11.296327,N/A,N/A
1,AAL,13.365,N/A,5.937208,N/A,-1.86,N/A,0.1802,N/A,4.861705,N/A,0.613111,N/A,N/A
2,AAP,62.200,N/A,19.963291,N/A,1.40,N/A,0.3288,N/A,8.410737,N/A,1.103824,N/A,N/A
3,AAPL,185.705,N/A,29.758598,N/A,46.45,N/A,7.5300,N/A,23.334671,N/A,17.357393,N/A,N/A
4,ABBV,162.955,N/A,45.508490,N/A,24.38,N/A,5.3500,N/A,13.613619,N/A,9.439832,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,128.365,N/A,24.026419,N/A,-4.42,N/A,5.1200,N/A,19.579001,N/A,8.883408,N/A,N/A
501,ZBH,122.640,N/A,53.921694,N/A,2.04,N/A,3.5100,N/A,16.048701,N/A,5.900416,N/A,N/A
502,ZBRA,250.130,N/A,28.168258,N/A,4.35,N/A,2.5800,N/A,17.557614,N/A,6.487390,N/A,N/A
503,ZION,42.470,N/A,4.629249,N/A,1.32,N/A,1.2900,N/A,3.535666,N/A,1.334324,N/A,N/A


In [36]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,130.670,N/A,30.642558,N/A,6.50,N/A,5.5600,N/A,24.146682,N/A,11.296327,N/A,N/A
1,AAL,13.365,N/A,5.937208,N/A,-1.86,N/A,0.1802,N/A,4.861705,N/A,0.613111,N/A,N/A
2,AAP,62.200,N/A,19.963291,N/A,1.40,N/A,0.3288,N/A,8.410737,N/A,1.103824,N/A,N/A
3,AAPL,185.705,N/A,29.758598,N/A,46.45,N/A,7.5300,N/A,23.334671,N/A,17.357393,N/A,N/A
4,ABBV,162.955,N/A,45.508490,N/A,24.38,N/A,5.3500,N/A,13.613619,N/A,9.439832,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,128.365,N/A,24.026419,N/A,-4.42,N/A,5.1200,N/A,19.579001,N/A,8.883408,N/A,N/A
501,ZBH,122.640,N/A,53.921694,N/A,2.04,N/A,3.5100,N/A,16.048701,N/A,5.900416,N/A,N/A
502,ZBRA,250.130,N/A,28.168258,N/A,4.35,N/A,2.5800,N/A,17.557614,N/A,6.487390,N/A,N/A
503,ZION,42.470,N/A,4.629249,N/A,1.32,N/A,1.2900,N/A,3.535666,N/A,1.334324,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [43]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
6,ABMD,381.020,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
28,ALXN,182.500,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,299.590,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
48,ATVI,94.420,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
88,CERN,94.920,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
121,CTXS,103.900,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,24.420,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
137,DISH,5.770,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,49.260,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [44]:
for column in [column_peRatio, column_priceToBook, column_priceToSales, column_ev_by_ebitda, column_ev_by_gp]:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace= True)


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [45]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [48]:
metrics = {
    column_peRatio: column_pePercentile,
    column_priceToBook: column_pbPercentile,
    column_priceToSales: column_psPercentile,
    column_ev_by_ebitda: column_ev_by_ebitda_percentile,
    column_ev_by_gp: column_ev_by_gp_percentile
}


for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,130.670,N/A,30.642558,0.69505,6.50,0.780198,5.5600,0.806931,24.146682,0.863366,11.296327,0.813861,N/A
1,AAL,13.365,N/A,5.937208,0.09703,-1.86,0.10495,0.1802,0.011881,4.861705,0.055446,0.613111,0.017822,N/A
2,AAP,62.200,N/A,19.963291,0.461386,1.40,0.227723,0.3288,0.041584,8.410737,0.20396,1.103824,0.059406,N/A
3,AAPL,185.705,N/A,29.758598,0.683168,46.45,0.976238,7.5300,0.881188,23.334671,0.851485,17.357393,0.942574,N/A
4,ABBV,162.955,N/A,45.508490,0.871287,24.38,0.962376,5.3500,0.8,13.613619,0.469307,9.439832,0.714851,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,128.365,N/A,24.026419,0.546535,-4.42,0.10099,5.1200,0.782178,19.579001,0.728713,8.883408,0.677228,N/A
501,ZBH,122.640,N/A,53.921694,0.90297,2.04,0.375248,3.5100,0.60396,16.048701,0.568317,5.900416,0.419802,N/A
502,ZBRA,250.130,N/A,28.168258,0.645545,4.35,0.657426,2.5800,0.520792,17.557614,0.641584,6.487390,0.469307,N/A
503,ZION,42.470,N/A,4.629249,0.085149,1.32,0.209901,1.2900,0.238614,3.535666,0.023762,1.334324,0.091089,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [49]:
from statistics import mean

for row in rv_dataframe.index:
    ratio_percentiles = []
    for metric in metrics.keys():
        ratio_percentiles.append(rv_dataframe.loc[row, metrics[metric]])

    rv_dataframe.loc[row, column_rv_score] = mean(ratio_percentiles)


rv_dataframe
        

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,130.670,N/A,30.642558,0.69505,6.50,0.780198,5.5600,0.806931,24.146682,0.863366,11.296327,0.813861,0.791881
1,AAL,13.365,N/A,5.937208,0.09703,-1.86,0.10495,0.1802,0.011881,4.861705,0.055446,0.613111,0.017822,0.057426
2,AAP,62.200,N/A,19.963291,0.461386,1.40,0.227723,0.3288,0.041584,8.410737,0.20396,1.103824,0.059406,0.198812
3,AAPL,185.705,N/A,29.758598,0.683168,46.45,0.976238,7.5300,0.881188,23.334671,0.851485,17.357393,0.942574,0.866931
4,ABBV,162.955,N/A,45.508490,0.871287,24.38,0.962376,5.3500,0.8,13.613619,0.469307,9.439832,0.714851,0.763564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,128.365,N/A,24.026419,0.546535,-4.42,0.10099,5.1200,0.782178,19.579001,0.728713,8.883408,0.677228,0.567129
501,ZBH,122.640,N/A,53.921694,0.90297,2.04,0.375248,3.5100,0.60396,16.048701,0.568317,5.900416,0.419802,0.574059
502,ZBRA,250.130,N/A,28.168258,0.645545,4.35,0.657426,2.5800,0.520792,17.557614,0.641584,6.487390,0.469307,0.586931
503,ZION,42.470,N/A,4.629249,0.085149,1.32,0.209901,1.2900,0.238614,3.535666,0.023762,1.334324,0.091089,0.129703


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [52]:
rv_dataframe.sort_values(column_rv_score, ascending=True, inplace=True)
rv_dataframe = rv_dataframe[:50]

rv_dataframe.reset_index(drop=True, inplace=True)
rv_dataframe

/var/folders/tt/ptcp60n908q2mwfffgwm6jlm0000gn/T/ipykernel_1808/1757294691.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rv_dataframe.sort_values(column_rv_score, ascending=True, inplace=True)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,FRC,3.510,N/A,0.418421,0.077228,0.0456,0.106931,0.0934,0.00198,0.270279,0.011881,0.094878,0.005941,0.040792
1,GM,35.535,N/A,4.946077,0.087129,0.6594,0.110891,0.2856,0.027723,1.325675,0.013861,0.936327,0.039604,0.055842
2,AAL,13.365,N/A,5.937208,0.09703,-1.8600,0.10495,0.1802,0.011881,4.861705,0.055446,0.613111,0.017822,0.057426
3,F,11.545,N/A,7.472398,0.112871,1.0400,0.152475,0.2643,0.019802,2.347591,0.015842,1.377598,0.09703,0.079604
4,C,52.070,N/A,8.104315,0.140594,0.5246,0.108911,0.6746,0.091089,4.393871,0.043564,0.710184,0.023762,0.081584
5,UNM,45.415,N/A,7.160745,0.108911,0.9178,0.134653,0.7208,0.108911,4.259477,0.037624,0.720840,0.027723,0.083564
6,UAL,40.525,N/A,5.103122,0.089109,1.6500,0.294059,0.2781,0.021782,3.379361,0.021782,0.522700,0.013861,0.088119
7,KSS,26.000,N/A,-20.840388,0.033663,0.7889,0.120792,0.1687,0.009901,9.044801,0.219802,1.229880,0.077228,0.092277
8,XRX,16.060,N/A,11.898189,0.229703,0.7082,0.114851,0.2797,0.023762,5.412472,0.073267,1.013098,0.045545,0.097426
9,SYF,37.290,N/A,6.731856,0.10297,1.2100,0.194059,0.7351,0.114851,4.429702,0.045545,0.981274,0.043564,0.100198


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [53]:
portfolio_input()

Enter the value of your portfolio: 1000000


In [56]:
position_size = float(portfolio_size)/len(rv_dataframe.index)

for row in rv_dataframe.index:
    rv_dataframe.loc[row, column_shares_to_buy] = math.floor(position_size/rv_dataframe.loc[row, column_price])

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,FRC,3.510,5698,0.418421,0.077228,0.0456,0.106931,0.0934,0.00198,0.270279,0.011881,0.094878,0.005941,0.040792
1,GM,35.535,562,4.946077,0.087129,0.6594,0.110891,0.2856,0.027723,1.325675,0.013861,0.936327,0.039604,0.055842
2,AAL,13.365,1496,5.937208,0.09703,-1.8600,0.10495,0.1802,0.011881,4.861705,0.055446,0.613111,0.017822,0.057426
3,F,11.545,1732,7.472398,0.112871,1.0400,0.152475,0.2643,0.019802,2.347591,0.015842,1.377598,0.09703,0.079604
4,C,52.070,384,8.104315,0.140594,0.5246,0.108911,0.6746,0.091089,4.393871,0.043564,0.710184,0.023762,0.081584
5,UNM,45.415,440,7.160745,0.108911,0.9178,0.134653,0.7208,0.108911,4.259477,0.037624,0.720840,0.027723,0.083564
6,UAL,40.525,493,5.103122,0.089109,1.6500,0.294059,0.2781,0.021782,3.379361,0.021782,0.522700,0.013861,0.088119
7,KSS,26.000,769,-20.840388,0.033663,0.7889,0.120792,0.1687,0.009901,9.044801,0.219802,1.229880,0.077228,0.092277
8,XRX,16.060,1245,11.898189,0.229703,0.7082,0.114851,0.2797,0.023762,5.412472,0.073267,1.013098,0.045545,0.097426
9,SYF,37.290,536,6.731856,0.10297,1.2100,0.194059,0.7351,0.114851,4.429702,0.045545,0.981274,0.043564,0.100198


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [60]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine= 'xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index= False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [61]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )


percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [65]:
column_formats ={
    'A': [column_ticker, string_template],
    'B': [column_price, dollar_template],
     'C': [column_shares_to_buy, integer_template],
     'D': [column_peRatio, float_template],
     'E': [column_pePercentile, percent_template],
     'F': [column_priceToBook, float_template],
     'G': [column_pbPercentile, percent_template],
     'H': [column_priceToSales, float_template],
     'I': [column_psPercentile, percent_template],
     'J': [column_ev_by_ebitda, float_template],
     'K': [column_ev_by_ebitda_percentile, percent_template],
     'L': [column_ev_by_gp, float_template],
     'M': [column_ev_by_gp_percentile, percent_template],
     'N': [column_rv_score, percent_template]
}

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25 , column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])



## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [66]:
writer.close()